# Laboratório 1 - Parte 1: Índice Invertido e Busca Booleana

## Aluno: Paulo Vinícius Soares
## 20/04/2018

### Introdução

O contexto desse laboratório se dá dentro da disciplina de Recuperação da Informação e Busca na Web da Universidade Federal de Campina Grande (UFCG), no período 2018.1.

Para a primeira parte, faremos os _imports_ necessários das bibliotecas de Python especializadas em análise de dados. Utilizaremos _pandas_, _numpy_ e _nltk_.

In [1]:
import pandas as pd
import numpy as np
import nltk as nl
from collections import defaultdict

In [2]:
DATABASE_PATH = '../database/noticias_estadao.csv'

A base de dados a ser utilizada nesse laboratório foi extraída a partir de um conjunto de notícias políticas coletadas no [Estadão Online](http://www.estadao.com.br/).

In [3]:
noticias_estadao = pd.read_csv(DATABASE_PATH)

Analisando o _dataframe_ podemos visualizar que este conta com três campos: **titulo** referente ao título da notícia, **conteudo** referente ao corpo da matéria e o **idNoticia** referente ao identificador único da notícia. Podemos ver o _dataframe_ abaixo:

In [4]:
noticias_estadao.head()

,titulo,conteudo,idNoticia
0,11 dos eleitores do País são filiados a legendas,Há porém variações regionais nesse fenômeno En...,7617
1,11 executivos integram 1º pedido de condenação...,CURITIBA A força-tarefa da Operação Lava Jato ...,412
2,11 executivos integram 1º pedido de condenação...,CURITIBA A força-tarefa da Operação Lava Jato ...,415
3,13 de deputados do PMDB quer romper com PT,O Estado ouviu 54 dos 74 deputados do PMDB em ...,6736
4,2014 começou em 2007,O estudo do Estadão Dados publicado ontem sobr...,7611


Visto que o _dataframe_ foi importado corretamente, as seções abaixo tratarão da construção do índice invertido e, em seguida, das funções de busca. Serão definidas três tipos de busca: Busca **AND**, **OR** e de **Um Termo**.

### Definição da função de Índice Invertido

Para a construção do Índice Invertido é necessário verificar a ocorrência dos termos em cada documento mapeando-a para uma estrutura de dados, comumente um dicionário. Para fazer essa separação das palavras utilizaremos a função *word_tokenize()* da biblioteca _nltk_. Além disso, todos os termos devem estar minúsculos.

In [5]:
def produz_tokens(df):
    inverted_index = defaultdict(set)
    for i, row in df.iterrows():
        tokens_titulos = (word.lower() for word in (nl.word_tokenize(row['titulo'])))
        
        for token in tokens_titulos:
            inverted_index[token].add(row['idNoticia'])
            
        tokens_conteudo = (word.lower() for word in (nl.word_tokenize(row['conteudo'])))
        for token in tokens_conteudo:
            inverted_index[token].add(row['idNoticia'])
        
    return inverted_index

In [6]:
dict = produz_tokens(noticias_estadao)

### Definição das funções de busca

As três funções de busca estão definidas abaixo. Para as buscas de um termo, um acesso direto ao dicionário retorna o desejado. Para as buscas utilizando **AND** e **OR**, as operações de conjunto são bastante úteis.

In [7]:
def um_termo(termo):
    return dict[termo]

In [8]:
def busca_and(termo1, termo2):
    ocorr_termo1 = set(dict[termo1])
    ocorr_termo2 = set(dict[termo2])
    
    return ocorr_termo1 & ocorr_termo2

In [9]:
def busca_or(termo1, termo2):
    ocorr_termo1 = set(dict[termo1])
    ocorr_termo2 = set(dict[termo2])
    
    return ocorr_termo1 | ocorr_termo2

In [10]:
def busca(termo1, operador, termo2):
    if(operador == "AND"):
        return busca_and(termo1, termo2)
    elif(operador == "OR"):
        return busca_or(termo1, termo2)
    else:
        raise Exception('Busca não suportada! Buscas suportadas: "AND" e "OR".')

In [11]:
def search(string):
    termos = string.split()
    if len(termos) == 1:
        return um_termo(termos[0].lower())
    else:
        return busca(termos[0].lower(), termos[1], termos[2].lower())

### Testes de sanidade e asserts

Com as funções já definidas e o _set_ de ocorrências construído, os testes abaixo visam garantir a corretude dos algoritmos escritos:

In [12]:
def sanity_check():
    result_busca = search("Campina AND Grande")
    expected = [1952, 4802, 1987, 6694, 5382, 1770, 2763, 1068, 5870, 2777, 1370, 2779]
    
    return len(result_busca) == len(expected) and sorted(result_busca) == sorted(expected)

In [13]:
assert(sanity_check())

In [14]:
print("Número de ocorrências encontradas usando o termo 'candidatos': ", len(search("candidatos")))
assert len(search("debate OR presidencial")) == 1770
assert len(search("debate AND presidencial")) == 201

assert len(search("presidenciáveis OR corruptos")) == 164
assert len(search("presidenciáveis AND corruptos")) == 0

assert len(search("Belo OR Horizonte")) == 331
assert len(search("Belo AND Horizonte")) == 242

Número de ocorrências encontradas usando o termo 'candidatos':  1395


### Bônus

Para implementar o algoritmo genérico para consultas conjuntivas é necessário salvar a frequência dos termos, então vamos definir uma classe em _Python_ que guarde esse atributo.

In [15]:
class Token:
    def __init__(self, term, postings, frequency):
        self.term = term
        self.postings = postings
        self.frequency = frequency
        
#     def __lt__(self, other):
#         return self.frequency < other.frequency
    
#     def __eq__(self, other):
#         return self.frequency == other.frequency
    
#     def __gt__(self, other):
#         return self.frequency > other.frequency

Agora, implementamos o algoritmo de interseção baseado na frequência. Primeiro ordenamos a lista, depois pegamos os _postings_ do primeiro termo e associamos à variável **result**; o restante é atribuído à **termos**. O laço fará a interseção entre os _postings_ do termo de menor frequência e dos demais até que não haja mais nenhum _posting_ a ser analisado.

In [16]:
def intersect(tokens):
    termos = sorted(tokens, key=lambda x: x.frequency, reverse=False)
    result = termos[0].postings
    termos = termos[1:]
    
    termos.append(-1)
    
    i = 0
    while termos[i] != -1 and len(result) != 0:
        result = result & termos[0].postings
        termos = termos[1:]
        i = len(termos)-1
    
    return result

Agora basta tratar a entrada para que esta trate a string corretamente:

In [17]:
def get_entrada(lista_strings):
    new_tokens = []
    for termo in lista_strings:
        new_tokens.append(Token(termo, dict[termo], len(dict[termo])))

    return intersect(new_tokens)

In [18]:
def busca_conjuntiva(string):
    termos = (word.lower() for word in string.split(" AND "))
    return get_entrada(termos)

### Asserts

Para os asserts anteriores, a busca funciona de forma semelhante passando nos testes e comprovando sua corretude.

In [19]:
assert len(busca_conjuntiva("Campina AND Grande")) == 12
assert len(busca_conjuntiva("debate AND presidencial")) == 201
assert len(busca_conjuntiva("presidenciáveis AND corruptos")) == 0
assert len(busca_conjuntiva("Belo AND Horizonte")) == 242